# TTC Data Analysis (Group Term Project)

Source: [City of Toronto’s Open Data Catalogue](https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#917dd033-1fe5-4ba8-04ca-f683eec89761)

Dataset: TTC Subway & SRT Train Service Delay Data

In [15]:
# raw links to dataset from catalogue page

links = {
    "metadata" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway%20Delays%20Metadata.xlsx",
    "codes" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway%20&%20SRT%20Log%20Codes.xlsx"
}

links["data"] = {
    "Jan 2014 - April 2017" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway%20&%20SRT%20Logs%20(Jan01_14%20to%20April30_17).xlsx",
    "May 2017" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway%20&%20SRT%20Logs%20(May%202017).xlsx",
    "June 2017" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/SubwayDelay201706.xlsx",
    "July 2017" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/SubwaySRTLogs201707.xlsx",
    "August 2017" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/SubwaySRTLogs201708.xlsx",
    "September 2017" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/SubwaySRTLogs201709.xlsx",
    "October 2017" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/SubwaySRTLogs201710.xlsx",
    "November 2017" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway_&_SRT_Logs_(November_2017).xlsx",
    "December 2017" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway_&_SRT_Logs_(December_2017).xlsx",
    "January 2018" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway_SRT_Logs(January%202018).xlsx",
    "February 2018" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway&SRT_Logs_February_2018.xlsx",
    "March 2018" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway&SRT_Logs_March_2018.xlsx",
    "April 2018" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway&SRT_Logs_April_2018.xlsx",
    "May 2018" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway&SRT_Logs_May_2018.xlsx",
    "June 2018" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway_SRT_Logs(June2018).xlsx",
    "July 2018" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway_SRT_Logs(July_2018).xlsx",
    "August 2018" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway_&_SRT_Logs_(August_2018).xlsx",
    "September 2018" : "https://www.toronto.ca/ext/open_data/catalog/data_set_files/Subway_&_SRT_Logs_(September_2018).xlsx"
}


In [16]:
# global import here

import os
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Section 1: Download, first clean, combine the data

Convert and save remote XLS files to CSV format to local directory.

There is no need to run this section every time, so execution depends on local data folder presense.

In [17]:
dataFolder = 'data'
           
def saveMetadataAndCodes():
    df = pd.read_excel(links['metadata'])
    df.to_csv(os.path.join(dataFolder, 'metadata.csv'), sep='\t', encoding='utf-8', index=False)
    
    df = pd.read_excel(links['codes'], usecols=[2,3])
    df.dropna(inplace=True)
    df.to_csv(os.path.join(dataFolder, 'codes_sub.csv'), sep='\t', encoding='utf-8', index=False)
    
    df = pd.read_excel(links['codes'], usecols=[6,7])
    df.dropna(inplace=True)
    df.to_csv(os.path.join(dataFolder, 'codes_srt.csv'), sep='\t', encoding='utf-8', index=False)
    
def saveAllYearsData():
    totalRows = 0
    with open(os.path.join(dataFolder, 'all_years.csv'), 'w+') as allDataFile:
        
        for link in list(links["data"].values()):            
            print("processing ", link)
            df = pd.read_excel(link)
            df.to_csv(allDataFile, sep='\t', encoding='utf-8', index=False, header=(totalRows == 0))
            totalRows = totalRows + len(df)
            
        print("saved rows: ", totalRows)
        allDataFile.close()

def downloadAndSaveData():
    saveMetadataAndCodes()
    saveAllYearsData()   
        
# don't need to run it every time on notebook, but you may manually re-run it see coments bellow 
if not os.path.exists(dataFolder):
    os.makedirs(dataFolder)
    downloadAndSaveData()

# uncomment to regenerate CSV files in local folder
# downloadAndSaveData()

#### Section Conclusion

Combining all information in one CSV file allow us 
* get full statistics information about data we processing for future cleaning and categorizing
* do timeline analysis by all period of data
* more convinient to work and apply different filters

Also 'saved rows:  97677' information give us first insight about size of all data we have.

### Section 2: First look and some statistics 

In [18]:
# just print metadata information
pd.read_table(os.path.join(dataFolder, 'metadata.csv'), index_col=0)

,Description,Example
Field Name,,
Date,Date (YYYY/MM/DD),2016-12-31 00:00:00
Time,Time (24h clock),01:59:00
Day,Name of the day of the week,Saturday
Station,TTC subway station name,Rosedale Station
Code,TTC delay code,MUIS
Min Delay,Delay (in minutes) to subway service,5
Min Gap,Time length (in minutes) between trains,9
Bound,Direction of train dependant on the line,N
Line,"TTC subway line i.e. YU, BD, SHP, and SRT",YU


In [19]:
# delays codes look

print("SUB codes information:")
codesSub = pd.read_table(os.path.join(dataFolder, 'codes_sub.csv'))
codesSub.head()
codesSub.describe()

# convert to dictionary for convinience, to use, for example codesSub.get('EUAC') later
codesSub = codesSub.set_index('SUB RMENU CODE').to_dict().get('CODE DESCRIPTION')
codesSub.keys()

print("SRT codes information:")
codesSrt = pd.read_table(os.path.join(dataFolder, 'codes_srt.csv'))
codesSrt.head()
codesSrt.describe()

# convert to dictionary for convinience, to use, for example codesSub.get('EUAC') later
codesSrt = codesSrt.set_index('SRT RMENU CODE').to_dict().get('CODE DESCRIPTION')
codesSrt.keys()

SUB codes information:


,SUB RMENU CODE,CODE DESCRIPTION
0,EUAC,Air Conditioning
1,EUAL,Alternating Current
2,EUATC,ATC RC&S Equipment
3,EUBK,Brakes
4,EUBO,Body


,SUB RMENU CODE,CODE DESCRIPTION
count,129,129
unique,129,129
top,PUMO,SCADA Related Problems
freq,1,1


dict_keys(['EUAC', 'EUAL', 'EUATC', 'EUBK', 'EUBO', 'EUCA', 'EUCD', 'EUCH', 'EUCO', 'EUDO', 'EUECD', 'EUHV', 'EULT', 'EULV', 'EUME', 'EUNEA', 'EUNT', 'EUO', 'EUOE', 'EUOPO', 'EUPI', 'EUSC', 'EUTL', 'EUTM', 'EUTR', 'EUTRD', 'EUVA', 'EUVE', 'EUYRD', 'MUATC', 'MUCL', 'MUD', 'MUDD', 'MUEC', 'MUESA', 'MUFM', 'MUFS', 'MUGD', 'MUI', 'MUIE', 'MUIR', 'MUIRS', 'MUIS', 'MULD', 'MUNOA', 'MUO', 'MUODC', 'MUPAA', 'MUPLA', 'MUPLB', 'MUPLC', 'MUPR1', 'MUSAN', 'MUSC', 'MUTD', 'MUTO', 'MUWEA', 'MUWR', 'PUATC', 'PUCBI', 'PUCSC', 'PUCSS', 'PUDCS', 'PUMEL', 'PUMO', 'PUMST', 'PUOPO', 'PUSAC', 'PUSBE', 'PUSCA', 'PUSCR', 'PUSEA', 'PUSI', 'PUSIO', 'PUSIS', 'PUSLC', 'PUSNT', 'PUSO', 'PUSRA', 'PUSSW', 'PUSTC', 'PUSTP', 'PUSTS', 'PUSWZ', 'PUSZC', 'PUTCD', 'PUTD', 'PUTDN ', 'PUTIJ', 'PUTIS', 'PUTNT', 'PUTO', 'PUTOE', 'PUTR', 'PUTS', 'PUTSC', 'PUTSM', 'PUTTC', 'PUTTP', 'PUTWZ', 'SUAE', 'SUAP', 'SUBT', 'SUCOL', 'SUDP', 'SUEAS', 'SUG', 'SUO', 'SUPOL', 'SUROB', 'SUSA', 'SUSP', 'SUUT', 'TUATC', 'TUCC', 'TUDOE', 'TUKEY'

SRT codes information:


,SRT RMENU CODE,CODE DESCRIPTION
0,ERAC,Air Conditioning
1,ERBO,Body
2,ERCD,Consequential Delay (2nd Delay Same Fault)
3,ERCO,Couplers
4,ERDB,Disc Brakes


,SRT RMENU CODE,CODE DESCRIPTION
count,71,71
unique,71,71
top,SRAE,Miscellaneous Other
freq,1,1


dict_keys(['ERAC', 'ERBO', 'ERCD', 'ERCO', 'ERDB', 'ERDO', 'ERHV', 'ERLT', 'ERLV', 'ERME', 'ERNEA', 'ERNT', 'ERO', 'ERPR', 'ERRA', 'ERTB', 'ERTC', 'ERTL', 'ERTR', 'ERVE', 'ERWA', 'ERWS', 'MRCL', 'MRD', 'MRDD', 'MREC', 'MRESA', 'MRFS', 'MRIE', 'MRLD', 'MRNOA', 'MRO', 'MRPAA', 'MRPLA', 'MRPLB', 'MRPLC', 'MRPR1', 'MRSAN', 'MRSTM', 'MRTO', 'MRUI', 'MRUIR', 'MRWEA', 'PREL', 'PRO', 'PRS', 'PRSA', 'PRSL', 'PRSO', 'PRSP', 'PRST', 'PRSW', 'PRTST', 'PRW', 'SRAE', 'SRAP', 'SRBT', 'SRCOL', 'SRDP', 'SREAS', 'SRO', 'SRSA', 'SRSP', 'SRUT', 'TRDOE', 'TRNIP', 'TRNOA', 'TRO', 'TRSET', 'TRST', 'TRTC'])

In [20]:
# data look

df = pd.read_csv(os.path.join(dataFolder, 'all_years.csv'), delimiter='\t')

df.describe()
df.isnull().sum()

df.head()

,Min Delay,Min Gap,Vehicle
count,97677.000000,97677.000000,97677.000000
mean,1.931980,2.842409,4026.119383
std,8.476457,8.513597,2373.891885
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,3001.000000
50%,0.000000,0.000000,5212.000000
75%,3.000000,5.000000,5566.000000
max,999.000000,999.000000,72537.000000


Date             0
Time             0
Day              0
Station          0
Code             1
Min Delay        0
Min Gap          0
Bound        21102
Line           400
Vehicle          0
dtype: int64

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
0,2014-01-01,00:21,Wednesday,VICTORIA PARK STATION,MUPR1,55,60,W,BD,5111
1,2014-01-01,02:06,Wednesday,HIGH PARK STATION,SUDP,3,7,W,BD,5001
2,2014-01-01,02:40,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0
3,2014-01-01,03:10,Wednesday,LANSDOWNE STATION,SUDP,3,8,W,BD,5116
4,2014-01-01,03:20,Wednesday,BLOOR STATION,MUSAN,5,10,S,YU,5386


### Section 3: Real stations names mapping

In [21]:
#based on file with clean names was made by us using "Levenshtein’s Distance" approach
station_names = pd.read_csv('data/station_names.csv')

df1 = pd.merge(left=df, right=station_names, how='left', left_on=df['Station'], right_on=station_names['data_station']).drop(columns=['Unnamed: 0','key_0','data_station'], axis=1)

df1 = df1.dropna(subset=['real_station']) 
#keep station only
df1 = df1[df1.location == 'Station']

df1.head()
df1.describe()

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,real_station,location,real_line
0,2014-01-01,00:21,Wednesday,VICTORIA PARK STATION,MUPR1,55,60,W,BD,5111,victoria park,Station,BD
1,2014-01-01,02:06,Wednesday,HIGH PARK STATION,SUDP,3,7,W,BD,5001,high park,Station,BD
2,2014-01-01,02:40,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0,sheppard,Station,NaN
3,2014-01-01,03:10,Wednesday,LANSDOWNE STATION,SUDP,3,8,W,BD,5116,lansdowne,Station,BD
4,2014-01-01,03:20,Wednesday,BLOOR STATION,MUSAN,5,10,S,YU,5386,bloor,Station,NaN


,Min Delay,Min Gap,Vehicle
count,86324.000000,86324.000000,86324.000000
mean,2.066656,3.048584,4362.077024
std,8.704314,8.782745,2154.139062
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,5028.000000
50%,0.000000,0.000000,5250.000000
75%,3.000000,6.000000,5606.000000
max,999.000000,999.000000,72537.000000


In [22]:
#loading map's coordinates for underlayed map picture 2000x1300

coords_columns = ['x', 'y', 'latitude', 'longitude', 'name']

coords1 = pd.read_csv('data/map/yonge-university.csv', delimiter=',', names=coords_columns)
coords2 = pd.read_csv('data/map/bloor-danforth.csv', delimiter=',', names=coords_columns)
coords3 = pd.read_csv('data/map/sheppard-yonge.csv', delimiter=',', names=coords_columns)
coords4 = pd.read_csv('data/map/srt.csv', delimiter=',', names=coords_columns)

coords = pd.DataFrame(columns=coords_columns)
coords = coords.append(coords1, ignore_index=True)
coords = coords.append(coords2, ignore_index=True)
coords = coords.append(coords3, ignore_index=True)
coords = coords.append(coords4, ignore_index=True)

#lowercase the station name for merge with previous DF
coords['name_lowercase'] = coords['name'].str.lower()

coords.head()

,x,y,latitude,longitude,name,name_lowercase
0,303,71,NaN,NaN,Vaughan Metropolitan Centre,vaughan metropolitan centre
1,303,105,NaN,NaN,Highway 407,highway 407
2,365,164,NaN,NaN,Pioneer Village,pioneer village
3,409,196,NaN,NaN,York University,york university
4,440,261,NaN,NaN,Finch West,finch west


In [23]:
df2 = pd.merge(left=df1, right=coords, how='left', left_on=df1['real_station'], right_on=coords['name_lowercase']).drop(columns=['key_0'], axis=1)
df2 = df2.drop(columns=['Station', 'Vehicle', 'Line', 'latitude', 'longitude', 'name_lowercase'])

df2 = df2.dropna(subset=['name']) 

df2.head()
df2.describe()

,Date,Time,Day,Code,Min Delay,Min Gap,Bound,real_station,location,real_line,x,y,name
0,2014-01-01,00:21,Wednesday,MUPR1,55,60,W,victoria park,Station,BD,1445,857,Victoria Park
1,2014-01-01,02:06,Wednesday,SUDP,3,7,W,high park,Station,BD,397,881,High Park
2,2014-01-01,02:40,Wednesday,MUNCA,0,0,NaN,sheppard,Station,NaN,921,357,Sheppard
3,2014-01-01,03:10,Wednesday,SUDP,3,8,W,lansdowne,Station,BD,539,881,Lansdowne
4,2014-01-01,03:20,Wednesday,MUSAN,5,10,S,bloor,Station,NaN,921,881,Bloor


,Min Delay,Min Gap
count,94859.000000,94859.000000
mean,2.065244,3.057106
std,8.656038,8.820118
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,3.000000,6.000000
max,999.000000,999.000000


### Section 4: Delay's map visualisation (Bokeh library)

In [24]:
from bokeh.layouts import layout
from bokeh.layouts import widgetbox
from bokeh.plotting import figure, show, output_file

from bokeh.embed import file_html

from bokeh.io import show
from bokeh.io import output_notebook 

from bokeh.models import Text
from bokeh.models import Plot
from bokeh.models import Slider
from bokeh.models import Circle
from bokeh.models import Range1d
from bokeh.models import CustomJS
from bokeh.models import HoverTool
from bokeh.models import LinearAxis
from bokeh.models import ColumnDataSource
from bokeh.models import SingleIntervalTicker

output_notebook()

Loading BokehJS ...

In [25]:
#preparing total delays per station

df3 = df2
df3['delays_total'] = df3.groupby('name')['name'].transform('count')

delay_mean = df3['Min Delay'].groupby([df3['name']]).mean()

def getDelayMean(name):
    return delay_mean[name]
    
#coeff for normalizing circles
k = 0.5 * df3.describe().delays_total['max'] / df3.describe().delays_total['min']

df3 = df3.groupby('name').last().reset_index()
df3['r'] = df3['delays_total'] / k
df3['delay_mean'] = df3['name'].apply(getDelayMean)

df3.head()
df3.describe()


,name,Date,Time,Day,Code,Min Delay,Min Gap,Bound,real_station,location,real_line,x,y,delays_total,r,delay_mean
0,Bathurst,2018-09-29,21:44,Saturday,MUIS,0,0,E,bathurst,Station,BD,730,881,870,7.294207,1.825287
1,Bay,2018-09-29,18:42,Saturday,MUI,5,9,W,bay,Station,BD,873,881,673,5.642530,2.408618
2,Bayview,2018-09-24,11:03,Monday,MUTD,3,8,E,bayview,Station,SHP,1063,357,390,3.269817,3.061538
3,Bessarion,2018-09-26,11:56,Wednesday,MUSC,0,0,W,bessarion,Station,SHP,1135,357,233,1.953506,3.201717
4,Bloor,2018-09-30,10:40,Sunday,SUAP,0,0,S,bloor,Station,NaN,921,881,2110,17.690549,2.133649


,Min Delay,Min Gap,x,y,delays_total,r,delay_mean
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,1.357143,2.542857,882.157143,737.028571,1355.128571,11.361596,2.298726
std,2.576571,4.551431,380.049029,250.230269,1516.759847,12.716737,0.961557
min,0.000000,0.000000,111.000000,105.000000,44.000000,0.368902,0.985573
25%,0.000000,0.000000,652.000000,548.000000,640.750000,5.372142,1.743096
50%,0.000000,0.000000,921.000000,881.000000,964.500000,8.086509,2.093568
75%,3.000000,5.000000,1063.000000,881.000000,1426.750000,11.962081,2.468735
max,12.000000,20.000000,1778.000000,1168.000000,10496.000000,88.000000,6.461271


In [26]:
hover = HoverTool(tooltips=[("Station", "@name"),("Delays count", "@delays_total"),("Delay mean", "@delay_mean")])

p = figure(x_range=(0,2000), y_range=(1333,0), width=980, height=600, x_axis_location="above", tools=[hover, 'save'])

#putting image wrom wikipedia as background
p.image_url(url=['data/map/TTC_subway_map_2018.png'], w=2000, h=1333, x=0, y=0)

p.axis.visible = False
# p.toolbar_location = None
p.grid.grid_line_color = None

source = ColumnDataSource(data=dict(
                        x=list(df3['x']), 
                        y=list(df3['y']),
                        r=list(df3['r']),
                        name=list(df3['name']),
                        delays_total=list(df3['delays_total']),
                        delay_mean=list(df3['delay_mean'])))

text_source = ColumnDataSource({'descr': ['%s' % 'Total delays']})
text = Text(x=1500, y=150, text='descr', text_font_size='34px', text_color='#AAAAAA')

p.add_glyph(text_source, text)
p.circle(x = df3['x'], y = df3['y'], size=7, color='white', alpha=0.9)
p.circle(x = 'x', y = 'y', size='r', source=source, alpha=0.4, line_color="#CC0C0C", fill_color="#FF0000", hover_color ="#FF0000", hover_alpha = 0.5)

show(p)

GlyphRenderer(id='1398', ...)

GlyphRenderer(id='1405', ...)

GlyphRenderer(id='1412', ...)

GlyphRenderer(id='1420', ...)

In [27]:
import datetime as dt

df4 = df2
df4['datetime'] = df4.Date + " " + df4.Time

df4 = df4.drop(columns=['Date', 'Time', 'delays_total','real_station','location'])
df4['datetime'] = pd.to_datetime(df4['datetime'], format='%Y-%m-%d %H:%M')

df4 = df4.set_index('datetime')

df4.describe()

sources = {}
yr = {}

def yearData(year):
    year = df4.loc[year+'-01-01': year+'-12-31 23:59']
    year['delays_total'] = year.groupby('name')['name'].transform('count')
    delay_mean1 = year['Min Delay'].groupby([year['name']]).mean()    
    
    def getDelayMean1(name):
        return delay_mean1[name]
    
    year = year.groupby('name').last().reset_index()
    year['r'] = year['delays_total'] / k/0.3
    year['delay_mean'] = year['name'].apply(getDelayMean1)

    return year

def sourcesData(year):
    sources['_'+year]= ColumnDataSource(yearData(year))

years = [2014,2015,2016,2017,2018]
years_dict = dict(zip([x for x in years], ['_%s' % x for x in years]))
years_js = str(years_dict).replace("'", "")

for y in years:
    sourcesData(str(y))
    

,Min Delay,Min Gap
count,94859.000000,94859.000000
mean,2.065244,3.057106
std,8.656038,8.820118
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,3.000000,6.000000
max,999.000000,999.000000


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
renderer_source = sources['_%s' % years[0]]
circle_glyph = Circle(x = 'x', y = 'y', size='r', fill_alpha=0.4, line_color="#CC0C0C", fill_color="#FF0000")

plot = figure(x_range=(0,2000), y_range=(1333,0), width=980, height=600, x_axis_location="above", tools=[hover, 'save'])

#putting image wrom wikipedia as background
plot.image_url(url=['data/map/TTC_subway_map_2018.png'], w=2000, h=1333, x=0, y=0)

plot.axis.visible = False
p.toolbar_location = None
plot.grid.grid_line_color = None

text_source = ColumnDataSource({'year': ['%s delays' % years[0]]})
text = Text(x=1550, y=150, text='year', text_font_size='34px', text_color='#AAAAAA')

plot.add_glyph(text_source, text)
plot.circle(x = df3['x'], y = df3['y'], size=7, color='white', alpha=0.9)

circle_renderer = plot.add_glyph(renderer_source, circle_glyph)


# Add the slider
code = """
    var year = slider.value,
        sources = %s,
        new_source_data = sources[year].data;
    renderer_source.data = new_source_data;
    text_source.data = {'year': [String(year) + ' delays']};
""" % years_js

callback = CustomJS(args=sources, code=code)
slider = Slider(start=years[0], end=years[-1], value=1, step=1, callback=callback)
callback.args["renderer_source"] = renderer_source
callback.args["slider"] = slider
callback.args["text_source"] = text_source

show(layout([[plot], [slider]], sizing_mode='scale_width'))

GlyphRenderer(id='1590', ...)

GlyphRenderer(id='1596', ...)

GlyphRenderer(id='1603', ...)